In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

In [26]:
a = np.load('/Users/dkorduban/workspace/sc2/my/replays/npz/a.npz')
batch = a['both']
print(batch.shape)
print(len(batch))

(46, 6, 30, 30)
46


In [7]:
batch = torch.Tensor(batch)

In [8]:
print(batch.shape)

torch.Size([46, 6, 30, 30])


In [ ]:
def ortho_weights(shape, scale=1.):
    """ PyTorch port of ortho_init from baselines.a2c.utils """
    shape = tuple(shape)

    if len(shape) == 2:
        flat_shape = shape[1], shape[0]
    elif len(shape) == 4:
        flat_shape = (np.prod(shape[1:]), shape[0])
    else:
        raise NotImplementedError

    a = np.random.normal(0., 1., flat_shape)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == flat_shape else v
    q = q.transpose().copy().reshape(shape)

    if len(shape) == 2:
        return torch.from_numpy((scale * q).astype(np.float32))
    if len(shape) == 4:
        return torch.from_numpy((scale * q[:, :shape[1], :shape[2]]).astype(np.float32))


def atari_initializer(module):
    """ Parameter initializer for Atari models

    Initializes Linear, Conv2d, and LSTM weights.
    """
    classname = module.__class__.__name__

    if classname == 'Linear':
        module.weight.data = ortho_weights(module.weight.data.size(), scale=np.sqrt(2.))
        module.bias.data.zero_()

    elif classname == 'Conv2d':
        module.weight.data = ortho_weights(module.weight.data.size(), scale=np.sqrt(2.))
        module.bias.data.zero_()

    elif classname == 'LSTM':
        for name, param in module.named_parameters():
            if 'weight_ih' in name:
                param.data = ortho_weights(param.data.size(), scale=1.)
            if 'weight_hh' in name:
                param.data = ortho_weights(param.data.size(), scale=1.)
            if 'bias' in name:
                param.data.zero_()


class ConvVAE(nn.Module):
    def __init__(self, z_dim):
        """ Basic convolutional variational autoencoder
        """
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(6, 32, 4, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 4, stride=2),
            nn.ReLU(inplace=True)
        )
        
        last_dim = 128 * 4

        self.mu = nn.Linear(last_dim, z_dim)
        self.logvar = nn.Linear(last_dim, z_dim)
        
        self.
        
    def forward(self, conv_in):
        """ Module forward pass

        Args:
            conv_in (Variable): convolutional input, shaped [N x 4 x 84 x 84]

        Returns:
            pi (Variable): action probability logits, shaped [N x self.num_actions]
            v (Variable): value predictions, shaped [N x 1]
        """
        N = conv_in.size()[0]

        conv_out = self.conv(conv_in).view(N, 64 * 7 * 7)

        fc_out = self.fc(conv_out)

        pi_out = self.pi(fc_out)
        v_out = self.v(fc_out)

        return pi_out, v_out

In [10]:
nn.Conv2d??

In [2]:
import torch.nn.modules.rnn as rnn

In [11]:
nn.RNNBase??

In [20]:
from torch.utils.data import Dataset,DataLoader

In [19]:
torch.utils.data.Dataset??

In [82]:
class MySimpleDataset(Dataset):
    def __init__(self, path):
        a = np.load(path)
        self._data = a['both']
    def __getitem__(self, index):
        return self._data[index]
    def __len__(self):
        return len(self._data)
    
dataset = MySimpleDataset('/Users/dkorduban/workspace/sc2/my/replays/npz/a.npz')
loader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)

In [22]:
a = np.array([1.,2])

In [23]:
a??

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from lagom.core.networks import make_fc
from lagom.core.networks import make_cnn
from lagom.core.networks import make_transposed_cnn

from lagom.core.networks import ortho_init

from lagom.core.networks import BaseVAE

class ConvVAE(BaseVAE):
    def make_encoder(self, config):
        out = make_cnn(input_channel=1, 
                       channels=[64, 64, 64], 
                       kernels=[4, 4, 4], 
                       strides=[2, 2, 1], 
                       paddings=[0, 0, 0])
        last_dim = 256
        
        return out, last_dim
        
    def make_moment_heads(self, config, last_dim):
        out = {}
        
        z_dim = config['network.z_dim']
        
        out['mu_head'] = nn.Linear(in_features=last_dim, out_features=z_dim)
        out['logvar_head'] = nn.Linear(in_features=last_dim, out_features=z_dim)
        out['z_dim'] = z_dim
        
        return out

    def make_decoder(self, config, z_dim):
        out = nn.ModuleList()
        
        out.append(nn.Linear(in_features=z_dim, out_features=self.last_dim))
        
        out.extend(make_transposed_cnn(input_channel=64, 
                                       channels=[64, 64, 64], 
                                       kernels=[4, 4, 4], 
                                       strides=[2, 1, 1], 
                                       paddings=[0, 0, 0], 
                                       output_paddings=[0, 0, 0]))
        
        out.append(nn.Linear(in_features=9216, out_features=28*28*1))
        
        return out

    def init_params(self, config):
        for layer in self.encoder:
            ortho_init(layer, nonlinearity='relu', constant_bias=0.0)
            
        ortho_init(self.mu_head, nonlinearity=None, weight_scale=0.01, constant_bias=0.0)
        ortho_init(self.logvar_head, nonlinearity=None, weight_scale=0.01, constant_bias=0.0)
        
        for layer in self.decoder:
            ortho_init(layer, nonlinearity='relu', constant_bias=0.0)

    def encoder_forward(self, x):
        for layer in self.encoder:
            x = F.relu(layer(x))

        return x

    def decoder_forward(self, z):
        # Forward of first fully-connected layer
        x = F.relu(self.decoder[0](z))
        
        # Reshape as [NxCxHxW]
        x = x.view(-1, 64, 2, 2)
        
        # Forward pass through transposed convolutional layer
        for layer in self.decoder[1:-1]:
            x = F.relu(layer(x))
            
        # Flatten to [N, D]
        x = x.flatten(start_dim=1)

        # Element-wise binary output
        x = torch.sigmoid(self.decoder[-1](x))

        return x

    
vae = ConvVAE(config={'network.z_dim': 32})

x = torch.randn(10, 1, 28, 28)
print(x.shape)
y = vae.encoder_forward(x)
print(y.shape)
y = y.flatten(start_dim=1)
print(y.shape)
# Forward pass through moment heads to obtain mu and logvar for latent variable
mu = vae.mu_head(y)
logvar = vae.logvar_head(y)
print(mu.shape, logvar.shape)
# Sample latent variable by using reparameterization trick
z = vae.reparameterize(mu, logvar)
print(z.shape)
# Forward pass through decoder of sampled latent variable to obtain reconstructed input
re_x = vae.decoder_forward(z)
print(re_x.shape)
# z = vae.reparameterize(y)
# print(z.shape)
# z = vae.decoder_forward(z)
# print(z.shape)

torch.Size([10, 1, 28, 28])
torch.Size([10, 64, 2, 2])
torch.Size([10, 256])
torch.Size([10, 32]) torch.Size([10, 32])
torch.Size([10, 32])
torch.Size([10, 784])


In [51]:
'''
self.encoder = nn.Sequential(
            nn.Conv2d(6, 32, 4, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 4, stride=2),
            nn.ReLU(inplace=True)
        )
        
        last_dim = 128 * 4

        self.mu = nn.Linear(last_dim, z_dim)
self.logvar = nn.Linear(last_dim, z_dim)
'''
x = torch.Tensor(batch)
print(x.shape)
x = nn.Conv2d(6, 32, 4, stride=2)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)
x = nn.Conv2d(32, 64, 4, stride=2)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)
x = nn.Conv2d(64, 128, 4, stride=2)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)
x = x.view(-1, 128 * 2 * 2)
print(x.shape)
x = nn.Linear(512, 64)(x)
print(x.shape)
x = nn.Linear(64, 512)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)
x = x.view(-1, 128, 2, 2)
print(x.shape)
x = nn.ConvTranspose2d(128, 64, 4, stride=2)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)
x = nn.ConvTranspose2d(64, 32, 4, stride=2)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)
x = nn.ConvTranspose2d(32, 6, 4, stride=2)(x)
print(x.shape)
x = nn.ReLU(inplace=True)(x)
print(x.shape)

torch.Size([46, 6, 30, 30])
torch.Size([46, 32, 14, 14])
torch.Size([46, 32, 14, 14])
torch.Size([46, 64, 6, 6])
torch.Size([46, 64, 6, 6])
torch.Size([46, 128, 2, 2])
torch.Size([46, 128, 2, 2])
torch.Size([46, 512])
torch.Size([46, 64])
torch.Size([46, 512])
torch.Size([46, 512])
torch.Size([46, 128, 2, 2])
torch.Size([46, 64, 6, 6])
torch.Size([46, 64, 6, 6])
torch.Size([46, 32, 14, 14])
torch.Size([46, 32, 14, 14])
torch.Size([46, 6, 30, 30])
torch.Size([46, 6, 30, 30])


In [72]:
class Sc2ConvVAE(BaseVAE):
    def make_encoder(self, config):
        out = make_cnn(input_channel=6, 
                       channels=[32, 64, 128], 
                       kernels=[4, 4, 4], 
                       strides=[2, 2, 2], 
                       paddings=[0, 0, 0])
        last_dim = 128 * 2 * 2
        
        return out, last_dim
        
    def make_moment_heads(self, config, last_dim):
        out = {}
        
        z_dim = config['network.z_dim']
        
        out['mu_head'] = nn.Linear(in_features=last_dim, out_features=z_dim)
        out['logvar_head'] = nn.Linear(in_features=last_dim, out_features=z_dim)
        out['z_dim'] = z_dim
        
        return out

    def make_decoder(self, config, z_dim):
        out = nn.ModuleList()
        
        out.append(nn.Linear(in_features=z_dim, out_features=self.last_dim))
        
        out.extend(make_transposed_cnn(input_channel=128, 
                                       channels=[64, 32, 6], 
                                       kernels=[4, 4, 4], 
                                       strides=[2, 2, 2], 
                                       paddings=[0, 0, 0], 
                                       output_paddings=[0, 0, 0]))
        return out

    def init_params(self, config):
        for layer in self.encoder:
            ortho_init(layer, nonlinearity='relu', constant_bias=0.0)
            
        ortho_init(self.mu_head, nonlinearity=None, weight_scale=0.01, constant_bias=0.0)
        ortho_init(self.logvar_head, nonlinearity=None, weight_scale=0.01, constant_bias=0.0)
        
        for layer in self.decoder:
            ortho_init(layer, nonlinearity='relu', constant_bias=0.0)

    def encoder_forward(self, x):
        for layer in self.encoder:
            x = F.relu(layer(x))

        return x

    def decoder_forward(self, z):
        # Forward of first fully-connected layer
        x = F.relu(self.decoder[0](z))
        
        # Reshape as [NxCxHxW]
        x = x.view(-1, 128, 2, 2)
        
        # Forward pass through transposed convolutional layer
        for layer in self.decoder[1:]:
            x = F.relu(layer(x))
            
#         # Flatten to [N, D]
#         x = x.flatten(start_dim=1)

#         # Element-wise binary output
#         x = torch.sigmoid(self.decoder[-1](x))

        return x

vae = Sc2ConvVAE(config={'network.z_dim': 64})
x = torch.Tensor(batch)
print(x.shape)
re_x, _, _ = vae.forward(x)
print(re_x.shape)
re_x[0][3]

torch.Size([46, 6, 30, 30])
torch.Size([46, 6, 30, 30])


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0014,
         0.0000, 0.0493, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0894, 0.0000, 0.0000, 0.0173, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0012, 0.0000, 0.0000],
        [0.0051, 0.0317, 0.0000, 0.0000, 0.0542, 0.0214, 0.0637, 0.0000, 0.0450,
         0.0000, 0.0000, 0.0000, 0.1060, 0.0000, 0.1060, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0045, 0.0064, 0.0068, 0.0376, 0.0000, 0.1211, 0.0000, 0.0037,
         0.0000, 0.0000, 0.0000],
        [0.0905, 0.0066, 0.0329, 0.0773, 0.1327, 0.0512, 0.0865, 0.0225, 0.0291,
         0.3268, 0.1661, 0.1018, 0.0715, 0.2939, 0.1133, 0.0000, 0.0000, 0.3505,
         0.1562, 0.0000, 0.0000, 0.1325, 0.0393, 0.1075, 0.0000, 0.2292, 0.1605,
         0.0099, 0.0511, 0.1018],
        [0.0234, 0.0082, 0.0000, 0.0073, 0.0000, 0.0777, 0.0000, 0.0000, 0.0306,
         0.0605, 0.0000, 0.0978, 0.0000, 0.0000, 0.0000, 0.0000, 0.0230, 0.0000,
       

In [88]:
from lagom import Logger

from lagom.engine import BaseEngine
import torch.optim as optim

from torchvision.utils import save_image


model = Sc2ConvVAE(config={'network.z_dim': 32})
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model.train()  # set to training mode

# Create a logger
train_output = Logger()

# Iterate over data batches for one epoch
for epoch in range(10):
    print('EPOCH', epoch)
    for i, data in enumerate(loader):
        # Put data to device
    #     data = data.to(self.device)
        # Zero-out gradient buffer
        optimizer.zero_grad()
        # Forward pass of data
        data = data / 300.
        re_x, mu, logvar = model(data)
        # Calculate loss
        out = model.calculate_loss(re_x=re_x, x=data, mu=mu, logvar=logvar, loss_type='MSE')
        loss = out['loss']
        # Backward pass to calcualte gradients
        loss.backward()
        # Take a gradient step
        optimizer.step()

        # Record train output
    #     train_output.log('epoch', n)
        train_output.log('iteration', i)
        train_output.log('train_loss', out['loss'].item())  # item() saves memory
        train_output.log('reconstruction_loss', out['re_loss'].item())
        train_output.log('KL_loss', out['KL_loss'].item())

        # Dump logging
        if i == 0 or (i+1) % 10 == 0:
            print('-'*50)
            train_output.dump(keys=None, index=-1, indent=0)
            print('-'*50)
        
    with torch.no_grad():  # fast, disable grad
        z = torch.randn(16, 32)
        re_x = model.decoder_forward(z)
        re_x = re_x.view(16, 6, 30, 30) * 5
        me, him = re_x.split(3, dim=1)
        save_image(me, f'sample_me_{epoch}.png')
        save_image(me, f'sample_him_{epoch}.png')
        

    # train_output.logs

#     def log_train(self, train_output, **kwargs):
#         logdir = kwargs['logdir']
#         epoch = kwargs['epoch']
        
#         mean_loss = np.mean(train_output['train_loss'])
#         print(f'====> Average loss: {mean_loss}')
        
#         # Use decoder to sample images from standard Gaussian noise
#         with torch.no_grad():  # fast, disable grad
#             z = torch.randn(64, self.config['network.z_dim']).to(self.device)
#             re_x = self.agent.decoder_forward(z).cpu()
#             save_image(re_x.view(64, 1, 28, 28), f'{logdir}/sample_{epoch}.png')

#     def eval(self, n=None):
#         self.agent.eval()  # set to evaluation mode
        
#         # Create a logger
#         eval_output = Logger()
        
#         # Iterate over test batches
#         for i, (data, label) in enumerate(self.test_loader):
#             # Put data to device
#             data = data.to(self.device)
#             with torch.no_grad():  # fast, disable grad
#                 # Forward pass of data
#                 re_x, mu, logvar = self.agent(data)
#                 # Calculate loss
#                 out = self.agent.calculate_loss(re_x=re_x, x=data, mu=mu, logvar=logvar, loss_type='BCE')
            
#             # Record eval output
#             eval_output.log('eval_loss', out['loss'].item())
        
#         return eval_output.logs

#     def log_eval(self, eval_output, **kwargs):
#         logdir = kwargs['logdir']
#         epoch = kwargs['epoch']
        
#         mean_loss = np.mean(eval_output['eval_loss'])
#         print(f'====> Test set loss: {mean_loss}')
        
#         # Reconstruct some test images
#         data, label = next(iter(self.test_loader))  # get a random batch
#         data = data.to(self.device)
#         n = min(data.size(0), 8)  # number of images
#         D = data[:n]
#         with torch.no_grad():  # fast, disable grad
#             re_x, _, _ = self.agent(D)
#         compare_img = torch.cat([D.cpu(), re_x.cpu().view(-1, 1, 28, 28)])
#         save_image(compare_img, f'{logdir}/reconstruction_{epoch}.png', nrow=n)

EPOCH 0
--------------------------------------------------
Iteration: 0
Train Loss: 49.87052917480469
Reconstruction Loss: 49.870521545410156
Kl Loss: 8.255243301391602e-06
--------------------------------------------------
--------------------------------------------------
Iteration: 9
Train Loss: 27.063880920410156
Reconstruction Loss: 26.904666900634766
Kl Loss: 0.15921440720558167
--------------------------------------------------
--------------------------------------------------
Iteration: 19
Train Loss: 27.50695037841797
Reconstruction Loss: 27.50146484375
Kl Loss: 0.005484968423843384
--------------------------------------------------
--------------------------------------------------
Iteration: 29
Train Loss: 26.453794479370117
Reconstruction Loss: 26.452856063842773
Kl Loss: 0.0009378194808959961
--------------------------------------------------
--------------------------------------------------
Iteration: 39
Train Loss: 67.16848754882812
Reconstruction Loss: 67.112701416015

--------------------------------------------------
Iteration: 9
Train Loss: 3.8887088298797607
Reconstruction Loss: 1.9025189876556396
Kl Loss: 1.986189842224121
--------------------------------------------------
--------------------------------------------------
Iteration: 19
Train Loss: 8.633472442626953
Reconstruction Loss: 6.891654968261719
Kl Loss: 1.7418177127838135
--------------------------------------------------
--------------------------------------------------
Iteration: 29
Train Loss: 3.4793293476104736
Reconstruction Loss: 1.4441964626312256
Kl Loss: 2.035132884979248
--------------------------------------------------
--------------------------------------------------
Iteration: 39
Train Loss: 5.383312702178955
Reconstruction Loss: 3.2429263591766357
Kl Loss: 2.1403863430023193
--------------------------------------------------
EPOCH 9
--------------------------------------------------
Iteration: 0
Train Loss: 10.085808753967285
Reconstruction Loss: 8.575077056884766
Kl L

In [79]:
optim.Adam??